<a href="https://colab.research.google.com/github/alerodriguessf/kmeans-clustering-clients-commerce/blob/main/Portfolio_Clustering_Model_Kmeans_20250122.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#Portfolio_Clustering Model_Kmeans_20250122

In [ ]:
#Dependency Installation

!pip install sidetable yellowbrick
!pip install ydata-profiling
!pip install --upgrade matplotlib


In [ ]:
#Importing libraries

import pandas as pd
import numpy as np
import sidetable

import seaborn as sns
import plotly.express as px
import matplotlib.pyplot as plt

from sklearn.cluster import KMeans, AgglomerativeClustering
from sklearn.preprocessing import StandardScaler, PowerTransformer
from sklearn.metrics import silhouette_score, davies_bouldin_score, calinski_harabasz_score
from sklearn.mixture import GaussianMixture
from ydata_profiling import ProfileReport

from yellowbrick.cluster import KElbowVisualizer


In [ ]:
from google.colab import files

# Upload the file
uploaded = files.upload()



## **Step 01) Exploratory Data Analysis**

1. Load the dataset;
2. Perform a statistical description of the data;
3. Visualize distributions and identify the relevance of columns for analysis;
4. Check for missing data, duplicates, outliers, and other inconsistencies

## 1.1 Loading the dataset;


In [ ]:
# Load the DataFrame

df = pd.read_csv('data.csv', encoding='latin1')

## 1.2 Performing a statistical description of the data;


In [ ]:
# Generating a statistical description of the data;
df.profile_report()

In [ ]:
# Generates an initial report to explore the data
df.head()

In [ ]:
# Assess if all variables are of the correct type to apply possible changes during the preprocessing stage
df.info()

In [ ]:
df.describe

## 1.3 Visualizing distributions and identifying the relevance of columns for analysis;

In [ ]:
#Identifying the number of unique values in two variables

df[['Description', 'Country']].nunique()

In [ ]:
# Identifying outliers, negative values, and other inconsistencies in the data

df[['Quantity','UnitPrice']].describe()

In [ ]:
# Visualizing the distribution frequency by country to evaluate whether the country column makes sense to keep

plt.figure(figsize=(12, 6))
sns.countplot(x=df['Country'], color='blue')
plt.title('Distribuição dos paises')
plt.xlabel('Paises')
plt.ylabel('Frequência')
plt.xticks(rotation=90)
plt.show()

In [ ]:
# A simpler representation of country frequency
df.stb.freq(['Country'])

In [ ]:
# Evaluate the correlation between numerical variables
corr = df.select_dtypes(include=np.number).corr()

sns.heatmap(corr, annot=True, cmap='coolwarm')
plt.show()

##1.4 Checking for missing data, duplicates, outliers, and other inconsistencies

### Although there is already a clear idea of the outliers based on the complete report from step 1

In [ ]:
# Plotted a boxplot to identify outliers in numerical variables
plt.figure(figsize=(12, 6))
sns.boxplot(df[['Quantity', 'UnitPrice']])
plt.title('Distribuição da quantidade e preço unitário')
plt.ylabel('Quantidade e preço unitário')
plt.xlabel('Frequência')
plt.show()


In [ ]:
# Sum of null rows in each column
df.isnull().sum().sort_values(ascending=False)

In [ ]:
# Checking for missing values
df.stb.missing()

In [ ]:
# Identifying duplicate rows

df.duplicated().sum()

## Step 02) Data Preprocessing

## 2.1 Remove empty rows

In [ ]:
# Remove all empty rows in the CustomerID field
df=df.dropna(subset=['CustomerID'])

In [ ]:
# Confirm the rows were removed
df.stb.missing()

## 2.2 Adjust column types to be compatible with their respective contents

In [ ]:
# Convert columns to data types compatible with their contents

df['CustomerID'] = df['CustomerID'].astype(int)
df['InvoiceDate'] = pd.to_datetime(df['InvoiceDate'], errors='coerce', format = '%m/%d/%Y %H:%M')
df['Country'] = df['Country'].astype('category')

In [ ]:
df = df.copy()

In [ ]:
# Confirming that the types of each variable are correct
df.info()

## 2.3 Removing outliers and inconsistent values



In [ ]:
# Removing negative values in columns
below = df[['Quantity','UnitPrice']].le(0).any(axis=1)
df = df[~below].copy()

In [ ]:
# Confirming that negative values were excluded
df[['Quantity','UnitPrice']].plot.box()

In [ ]:
# Removing extreme outliers with reasonable limits
df = df.query('Quantity <10_000 & UnitPrice <8_000')

In [ ]:
df.info()

## 2.4 Calculating the RFM (Recency, Frequency and monetary)


In [ ]:
# Creating a new column for the total purchase value
df['price_total'] = df.Quantity * df.UnitPrice

In [ ]:
# Calculating the RFM
df_rfm = (
    df.groupby('CustomerID')
    .agg(
        recency=('InvoiceDate', lambda x: (pd.Timestamp('2012-01-01') - x.max()).days),
        frequency=('InvoiceNo', 'nunique'),
        monetary=('price_total', 'mean')
    )
)

df_rfm.head()

In [ ]:
# Assessing the distribution of the newly created columns
df_rfm.plot.box()

## 2.5 With the DataFrame adjusted for RFM values, I will standardize the data and repeat the process of removing outliers and inconsistent values


In [ ]:
from sklearn.preprocessing import scale

In [ ]:
# Standardize the data
df_rfm.apply(scale).plot.box()

In [ ]:
# Identifying outliers
df_rfm.apply(scale).query('monetary > 50')

In [ ]:
# Check inconsistencies in this outlier
df.query('CustomerID == 15098')

In [ ]:
# Remove the outlier
df_rfm = df_rfm.drop(15098)

In [ ]:
# Although the data will be normalized, it is already possible to identify outliers in the DataFrame
df_rfm.describe()

In [ ]:
# Plotted a graph to visually represent outliers
df_rfm.apply(scale).plot.box()

In [ ]:
# Removing outliers by replacing values above the 95th percentile with the 95th percentile value
df_rfm_clip = df_rfm.apply(lambda x: x.clip(upper = x.quantile(0.95)))

In [ ]:
# Confirming that outliers were removed
df_rfm_clip.describe()

In [ ]:
scaler = PowerTransformer()

In [ ]:
# Removing outliers using PowerTransformer
df_rfm_scaled = pd.DataFrame(scaler.fit_transform(df_rfm), index=df_rfm.index, columns=df_rfm.columns)

In [ ]:
# Normalizing cleaned data
df_rfm_scaled.head()

In [ ]:
# Visually representing normalized data
df_rfm_scaled.plot.box()

In [ ]:
# Creating a DataFrame with scaled and outlier-free data for clustering
df_rfm_clip_scaled = df_rfm_clip.apply(scale)

## Step 03) Select a clustering algorithm

1. Choose a suitable algorithm for the dataset, such as KMeans, DBSCAN, Hierarchical, or Mean Shift
2. Find the optimal number of clusters using the Elbow or Silhouette Score methods
3. Implement the chosen algorithm


## 3.1 Choosing KMeans to determine the optimal number of clusters

#### **I chose to use K-Means to define the number of clusters** because it is a simple, efficient method well-suited to this type of analysis and the dataset used in the challenge. K-Means works well for identifying approximately spherical and convex clusters, as seen in the context of RFM analysis (Recency, Frequency, and Monetary), where groups tend to follow this characteristic. K-Means is flexible, handles outliers well, and is easily interpretable.


In [ ]:
from yellowbrick.cluster import KElbowVisualizer

## 3.2 Finding the optimal number of clusters using the Elbow and Silhouette Score methods

In [ ]:
## Identifying the ideal number of clusters using the Elbow method

model = KMeans()
visualizer = KElbowVisualizer(model, k=(2,8), timings=False)
visualizer.fit(df_rfm_clip_scaled)
visualizer.show()

In [ ]:
# Evaluating the optimal number of clusters based on key clustering quality indicators
cluster_metrics = silhouette_score, davies_bouldin_score, calinski_harabasz_score
cluster_metrics_results = []
x = df_rfm_clip_scaled.copy()

for k in range(2, min(x.shape[0], 11)):
    model = KMeans(n_clusters=k, random_state=0)
    labels = model.fit_predict(x)
    cluster_results_dict = {'k': k}
    cluster_results_dict['inertia'] = model.inertia_
    for metric in cluster_metrics:
        cluster_results_dict[metric.__name__] = metric(x, labels)
    cluster_metrics_results.append(cluster_results_dict)

cluster_metrics_results

pd.DataFrame(cluster_metrics_results).set_index('k').style.background_gradient()

## After analyzing the metrics, everything seems to indicate that **4 clusters offer the best balance between cohesion and separation of groups**

###**Inertia**: The significant reduction in inertia stabilizes after k=4, meaning more clusters add little improvement.

###**Silhouette Score**: The maximum value at k=4 (0.479) shows good cohesion and separation between clusters.

###**Davies-Bouldin Score**: The lowest value occurs at k=4 (0.704), suggesting an optimal separation.

###**Calinski-Harabasz Index**: The highest value at k=4 (3751) highlights greater distinction between clusters.

## 3.3 Implementing the chosen algorithm


## 3.3 Implementing the chosen algorithm


In [ ]:
# Training the model with the optimal number of clusters
kmeans = KMeans(4)
kmeans_labels = kmeans.fit_predict(df_rfm_clip_scaled)

In [ ]:
# 3D visual representation of the clusters
px.scatter_3d(df_rfm_clip, x='recency', y='frequency', z='monetary', color=kmeans_labels, template='plotly_dark')

## Step 04) Analyze the clusters obtained from KMeans

1. Identify patterns and common characteristics among customers
2. Plot graphs to assist in the analysis

## 4.1 Identifying patterns and common characteristics among customers

In [ ]:
#Presenting the profiles of each cluster based on the RFM metric

df_rfm_clip['cluster'] = kmeans.labels_

cluster_profiles = df_rfm_clip.groupby('cluster').mean()
print("Perfis dos Clusters:\n", cluster_profiles)

In [ ]:
#Creating a DataFrame with normalized cluster data
centers = pd.DataFrame(kmeans.cluster_centers_, columns=df_rfm_clip_scaled.columns)
centers

In [ ]:
cluster_profiles_z = (cluster_profiles - cluster_profiles.mean()) / cluster_profiles.std()
print("Z-Scores dos Clusters:\n", cluster_profiles_z)


## 4.2 Plotting graphs to assist in the analysis

In [ ]:
sns.heatmap(cluster_profiles_z.T, annot=True, cmap="coolwarm", fmt=".2f")
plt.title("Divergência dos Clusters em Relação à Média Global")
plt.show()


In [ ]:
for col in ["recency", "frequency", "monetary"]:
    sns.boxplot(x="cluster", y=col, data=df_rfm_clip.reset_index())
    plt.title(f"Distribuição de {col} por Cluster")
    plt.show()

In [ ]:
#Visual representation of normalized cluster data

fig,axes = plt.subplots(nrows=4, figsize=(14,12), sharex=True)

for i,ax in enumerate(axes):
  center = centers.loc[i,:]
  maxPC = 1.01 * center.abs().max()
  colors = ['green' if l > 0 else 'red' for l in center]
  center.plot.bar(ax=ax, color=colors)
  ax.set_ylabel(f'Cluster {i+1}')
  ax.set_ylim(-maxPC, maxPC)
  ax.axhline(color='gray')
  ax.xaxis.set_ticks_position('none')

plt.xticks(rotation=60, ha='right')
plt.tight_layout()
plt.show()

In [ ]:
# Visual representation of clusters based on RFM indicators

(
 df_rfm_clip.assign(cluster= kmeans_labels)
 .groupby('cluster')
 .mean()
 .transpose()
 .style.background_gradient(cmap='YlOrRd', axis=1)
)

## Step 05) Interpret the results obtained

1. Describe the purchasing profile of customers in each cluster
2. Justify how this analysis can be useful for the company to segment its customers and personalize marketing campaigns
3. Suggest possible actions based on the analyses performed


5.1 Description of Customer Purchasing Profile by Cluster

**Cluster 1: Inactive Low-Value Customers**

Recency: 260 days (customers who haven’t purchased for a long time).

Frequency: 1.5 purchases on average (infrequent customers).

Monetary: $18.10 on average (very low average spend).

Profile: Sporadic customers, low retention, and little-added value.

**Cluster 2: Moderately Active Low-Value Customers**

Recency: 67 days (more recently engaged customers).

Frequency: 2.7 purchases on average (moderate frequency customers).

Monetary: $17.12 on average (low average spend).

Profile: Customers who make occasional purchases but still have low financial impact.

**Cluster 3: Active High-Value Customers**

Recency: 121 days (relatively active but not recent customers).

Frequency: 3.9 purchases on average (frequent customers).

Monetary: $80.71 on average (high average spend per customer).

Profile: Regular customers with high value for the company.

**Cluster 4: Recent and Very Frequent Customers**

Recency: 41 days (recent customers).

Frequency: 10.2 purchases on average (highly frequent customers).

Monetary: $20.43 on average (moderate average spend).

Profile: Engaged customers who buy frequently but with relatively low ticket sizes.

##5.2 Justifying how this analysis can be useful for the company to segment its customers and personalize marketing campaigns

**Segmentation**: The analysis allows targeted campaigns for each profile, optimizing resources and increasing effectiveness.

**Personalization**: Identifying needs and behavioral patterns helps create personalized offers and improve the customer experience.

**Retention**: Focus on higher-value clusters (2 and 3) to retain these customers and increase their lifecycle value.


## 5.3 Suggesting possible actions based on the analyses performed

**Cluster 1:** Inactive Low-Value Customers

Strategy: Reactivation campaigns, such as personalized discount emails or suggested products.

Incentive: Offer exclusive promotions for the next purchase.

**Cluster 2:** Moderately Active Low-Value Customers

Strategy: Increase spend with product packages or combos.

Incentive: Offer a basic loyalty program to encourage more frequent purchases.

**Cluster 3:** Active High-Value Customers

Strategy: Retain and increase engagement through VIP programs or exclusive offers.

Incentive: Create an exclusive communication channel for these customers with benefits like early access to launches.

**Cluster 4:** Recent and Very Frequent Customers

Strategy: Encourage higher-value purchases to increase average ticket size.

Incentive: Offer promotions based on purchase volume or recommend complementary products.
